<a href="https://colab.research.google.com/github/necroJADE/DataAnalytics/blob/main/SENTI_ANAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
data1 = pd.read_csv('/content/drive/MyDrive/DATASET/SENTI/training.1600000.processed.noemoticon.csv', encoding =DATASET_ENCODING , names=DATASET_COLUMNS)
data = data1.sample(frac=0.10)
data.head()
X = data.iloc[:,[5]]
Y = data.iloc[:,0]
Y[Y == 4] = 1

<ipython-input-4-5de3d89a1de2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[Y == 4] = 1


In [ ]:
data.head()

,target,ids,date,flag,user,text
474733,0,2177407241,Mon Jun 15 06:03:08 PDT 2009,NO_QUERY,jiaenn_,snip snip snip and my hair's significantly sho...
757078,0,2295199395,Tue Jun 23 07:36:53 PDT 2009,NO_QUERY,jeradmc,they should be streaming Wimbledon for free on...
1116978,1,1973332030,Sat May 30 10:56:41 PDT 2009,NO_QUERY,Hazar_Jast,Eating crawfish at Pat O'briens on Bourbon St.
356354,0,2044252609,Fri Jun 05 09:05:22 PDT 2009,NO_QUERY,laurensabine,@ronzo Aw.....give Otto an extra hug from us.
1055424,1,1962206003,Fri May 29 10:41:16 PDT 2009,NO_QUERY,nataliemelanson,Almost the weekend yipee!! Out to the movies ...


In [ ]:
data.head()

,target,ids,date,flag,user,text
474733,0,2177407241,Mon Jun 15 06:03:08 PDT 2009,NO_QUERY,jiaenn_,snip snip snip and my hair's significantly sho...
757078,0,2295199395,Tue Jun 23 07:36:53 PDT 2009,NO_QUERY,jeradmc,they should be streaming Wimbledon for free on...
1116978,1,1973332030,Sat May 30 10:56:41 PDT 2009,NO_QUERY,Hazar_Jast,Eating crawfish at Pat O'briens on Bourbon St.
356354,0,2044252609,Fri Jun 05 09:05:22 PDT 2009,NO_QUERY,laurensabine,@ronzo Aw.....give Otto an extra hug from us.
1055424,1,1962206003,Fri May 29 10:41:16 PDT 2009,NO_QUERY,nataliemelanson,Almost the weekend yipee!! Out to the movies ...


In [ ]:
# Text-preprocessing

# Missing Values
num_missing_desc = data.isnull().sum()[2]    # No. of values with msising descriptions
print('Number of missing values: ' + str(num_missing_desc))
data = data.dropna()

TAG_CLEANING_RE = "@\S+"
# Remove @tags
X['text'] = X['text'].map(lambda x: re.sub(TAG_CLEANING_RE, ' ', x))

# Smart lowercase
X['text'] = X['text'].map(lambda x: x.lower())

# Remove numbers
X['text'] = X['text'].map(lambda x: re.sub(r'\d+', ' ', x))

# Remove links
TEXT_CLEANING_RE = "https?:\S+|http?:\S|[^A-Za-z0-9]+"
X['text'] = X['text'].map(lambda x: re.sub(TEXT_CLEANING_RE, ' ', x))

# Remove Punctuation
X['text']  = X['text'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

# Remove white spaces
X['text'] = X['text'].map(lambda x: x.strip())

# Tokenize into words
X['text'] = X['text'].map(lambda x: word_tokenize(x))

# Remove non alphabetic tokens
X['text'] = X['text'].map(lambda x: [word for word in x if word.isalpha()])

# Filter out stop words
stop_words = set(stopwords.words('english'))
X['text'] = X['text'].map(lambda x: [w for w in x if not w in stop_words])

# Word Lemmatization
lem = WordNetLemmatizer()
X['text'] = X['text'].map(lambda x: [lem.lemmatize(word,"v") for word in x])

# Turn lists back to string
X['text'] = X['text'].map(lambda x: ' '.join(x))

Number of missing values: 0


In [ ]:
X.head()

,text
474733,snip snip snip hair significantly shorter
757078,stream wimbledon free online
1116978,eat crawfish pat briens bourbon st
356354,aw give otto extra hug us
1055424,almost weekend yipee movies tonight hali tomor...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("TRAIN size:", len(X_train))
print("TEST size:", len(X_train))

TRAIN size: 128000
TEST size: 128000


In [ ]:
import gensim

# WORD2VEC
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

documents = [_text.split() for _text in X_train.text]
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, workers=8)
w2v_model.build_vocab(documents)

In [ ]:
words = w2v_model.wv
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 6294


In [ ]:
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)


(23237038, 28890400)

In [ ]:
w2v_model.wv.most_similar("great")

[('wonderful', 0.47823357582092285),
 ('fantastic', 0.442758709192276),
 ('good', 0.4325845241546631),
 ('awesome', 0.3995739817619324),
 ('fabulous', 0.38894277811050415),
 ('nice', 0.34676292538642883),
 ('amaze', 0.3428371250629425),
 ('incredible', 0.33877068758010864),
 ('lovely', 0.3376127779483795),
 ('excellent', 0.30754008889198303)]

In [ ]:
w2v_model.wv.most_similar("great")

[('wonderful', 0.47823357582092285),
 ('fantastic', 0.442758709192276),
 ('good', 0.4325845241546631),
 ('awesome', 0.3995739817619324),
 ('fabulous', 0.38894277811050415),
 ('nice', 0.34676292538642883),
 ('amaze', 0.3428371250629425),
 ('incredible', 0.33877068758010864),
 ('lovely', 0.3376127779483795),
 ('excellent', 0.30754008889198303)]

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.utils.np_utils import to_categorical

# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 300

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.text)
word_index = tokenizer.word_index
vocab_size = len(word_index)
print('Found %s unique tokens.' % len(word_index))

# Convert the data to padded sequences
X_train_padded = tokenizer.texts_to_sequences(X_train.text)
X_train_padded = pad_sequences(X_train_padded, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train_padded.shape)

Found 54997 unique tokens.
Shape of data tensor: (128000, 300)


In [ ]:
# Embedding matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size+1, W2V_SIZE))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(54998, 300)


In [ ]:
# Build Model
from keras.layers import Conv1D, GlobalMaxPooling1D
import keras

model = Sequential()
model.add(Embedding(vocab_size+1, W2V_SIZE, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(Dropout(0.3))
model.add(Conv1D(300,3,activation='relu'))
# we use max pooling:
model.add(GlobalMaxPooling1D())
model.add(Dense(200, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          16499400  
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 conv1d (Conv1D)             (None, 298, 300)          270300    
                                                                 
 global_max_pooling1d (Globa  (None, 300)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 200)               60200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                        

In [ ]:
# Training
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

BATCH_SIZE = 1024
history = model.fit(X_train_padded, y_train,
                    batch_size=512,
                    epochs=5,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/5
225/225 [==============================] - ETA: 0s - loss: 0.5522 - accuracy: 0.7129

225/225 [==============================] - 1052s 5s/step - loss: 0.5522 - accuracy: 0.7129 - val_loss: 0.5044 - val_accuracy: 0.7498 - lr: 0.0010
Epoch 2/5
225/225 [==============================] - ETA: 0s - loss: 0.5092 - accuracy: 0.7462

225/225 [==============================] - 1082s 5s/step - loss: 0.5092 - accuracy: 0.7462 - val_loss: 0.4987 - val_accuracy: 0.7530 - lr: 0.0010
Epoch 3/5
225/225 [==============================] - ETA: 0s - loss: 0.4947 - accuracy: 0.7562

225/225 [==============================] - 1048s 5s/step - loss: 0.4947 - accuracy: 0.7562 - val_loss: 0.4978 - val_accuracy: 0.7548 - lr: 0.0010
Epoch 4/5
225/225 [==============================] - ETA: 0s - loss: 0.4820 - accuracy: 0.7635

225/225 [==============================] - 1081s 5s/step - loss: 0.4820 - accuracy: 0.7635 - val_loss: 0.4951 - val_accuracy: 0.7574 - lr: 0.0010
Epoch 5/5
225/225 [==============================] - ETA: 0s - loss: 0.4699 - accuracy: 0.7726

225/225 [==============================] - 1083s 5s/step - loss: 0.4699 - accuracy: 0.7726 - val_loss: 0.5040 - val_accuracy: 0.7536 - lr: 0.0010


In [ ]:
# Evaluation
import matplotlib.pyplot as plt
X_test_padded = tokenizer.texts_to_sequences(X_test.text)
X_test_padded = pad_sequences(X_test_padded, maxlen=MAX_SEQUENCE_LENGTH)
score = model.evaluate(X_test_padded, y_test, batch_size=512)
print("ACCURACY:",score[1])
print("LOSS:",score[0])

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

63/63 [==============================] - 112s 2s/step - loss: 0.5032 - accuracy: 0.7544
ACCURACY: 0.7543749809265137
LOSS: 0.5032249093055725


AttributeError: ignored

In [ ]:
import pickle

In [ ]:
model.save('drive/MyDrive/NBs/Sentiment_LSTM_model.h5')
with open('drive/MyDrive/NBs/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

AttributeError: ignored

In [ ]:
# Load Model
from keras.models import load_model
model = load_model('drive/MyDrive/NBs/Sentiment_LSTM_model.h5')
# loading tokenizer
with open('drive/MyDrive/NBs/trainHistoryDict', 'rb') as file_pi:
    history = pickle.load(file_pi)

In [ ]:
# Evaluation
import matplotlib.pyplot as plt
X_test_padded = tokenizer.texts_to_sequences(X_test.text)
X_test_padded = pad_sequences(X_test_padded, maxlen=MAX_SEQUENCE_LENGTH)
score = model.evaluate(X_test_padded, y_test, batch_size=512)
print("ACCURACY:",score[1])
print("LOSS:",score[0])

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

63/63 [==============================] - 106s 2s/step - loss: 0.5032 - accuracy: 0.7544
ACCURACY: 0.7543749809265137
LOSS: 0.5032249093055725


AttributeError: ignored

In [ ]:
def predict(text, include_neutral=True):
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    if(score >=0.4 and score<=0.6):
        label = "Neutral"
    if(score <=0.4):
        label = "Negative"
    if(score >=0.6):
        label = "Positive"

    return {"label" : label,
        "score": float(score)}

In [ ]:
predict("God is good")

1/1 [==============================] - 0s 89ms/step


{'label': 'Positive', 'score': 0.7301880717277527}

In [ ]:
df=predict("God is good")

1/1 [==============================] - 0s 44ms/step


In [ ]:
df


{'label': 'Positive', 'score': 0.7301880717277527}

In [ ]:
DATA_COLUMNS = ["Tweets"]
DATASET_ENCODING = "ISO-8859-1"
TD = pd.read_csv('/content/drive/MyDrive/DATASET/SENTI/Scraped/trail1.csv', encoding =DATASET_ENCODING , names=DATASET_COLUMNS)

In [ ]:
TD